In [2]:
%load_ext autoreload
import datetime
# import ete3
import itertools
import json
import logging
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import typing
# import re
import xml.etree.ElementTree as ET
# import xml.dom.minidom
# import xmlschema

In [3]:
%autoreload
from maxes.xes_loader2 import XesLoader, XesLog
# from maxes.xes_file import XesFile
# from maxes.analyze_xes import AnalyzeXes
# import maxes.analyze_sequence
# import maxes.serialization.serialize
# import maxes.graphs
import maxes.notebooks.utils
import maxes.utils

In [ ]:
maxes.notebooks.utils.init_notebook() # RUN ONLY ONCE

In [5]:
file_path = maxes.notebooks.utils.get_data_path("other/simple.xes")
log = XesLoader().load(file_path)
log.df

,case:concept:name,concept:name,lifecycle:transition,time:timestamp,call centre,org:resource,location,duration,outcome,case:description
0,1,incoming claim,complete,1970-01-01 00:00:00+00:00,Brisbane,customer,NaN,NaN,NaN,Simulated process instance
1,1,B check if sufficient information is available,start,1970-01-01 00:00:00+00:00,NaN,Call Centre Agent Brisbane,Brisbane,NaN,NaN,Simulated process instance
2,1,B check if sufficient information is available,complete,1970-01-01 00:00:41+00:00,NaN,Call Centre Agent Brisbane,Brisbane,NaN,NaN,Simulated process instance
3,1,B register claim,start,1970-01-01 00:00:41+00:00,NaN,Call Centre Agent Brisbane,NaN,NaN,NaN,Simulated process instance
4,1,B register claim,complete,1970-01-01 00:12:59+00:00,NaN,Call Centre Agent Brisbane,NaN,NaN,NaN,Simulated process instance
...,...,...,...,...,...,...,...,...,...,...
4,3512,S register claim,complete,1970-01-01 02:39:31+00:00,NaN,Call Centre Agent Sydney,NaN,NaN,NaN,Simulated process instance
5,3512,determine likelihood of claim,start,1970-01-01 06:56:04+00:00,NaN,Claims handler,NaN,NaN,NaN,Simulated process instance
6,3512,determine likelihood of claim,complete,1970-01-01 06:56:35+00:00,NaN,Claims handler,NaN,NaN,NaN,Simulated process instance
7,3512,end,start,1970-01-01 06:56:35+00:00,NaN,Claims handler,NaN,17306,not liable,Simulated process instance


In [29]:
%autoreload
from maxes.generators.xes_generator.xes_generator3 import XesGenerator3 as XesGenerator

generator = XesGenerator().fit(log)

In [30]:
generated_log = generator.generate()
generated_log

call centre
org:resource
time:timestamp
lifecycle:transition
concept:name
location
duration
outcome


In [31]:
generated_log.df

,case:concept:name,concept:name,lifecycle:transition,time:timestamp,call centre,location,duration,outcome,org:resource
0,0,incoming claim,complete,1970-01-01 07:19:46.536844+00:00,Sydney,NaN,NaN,NaN,customer
1,0,S check if sufficient information is available,start,1970-01-01 07:19:45.346105+00:00,NaN,Sydney,NaN,NaN,Call Centre Agent Sydney
2,0,S check if sufficient information is available,complete,1970-01-01 07:20:26.790099+00:00,NaN,Sydney,NaN,NaN,Call Centre Agent Sydney
3,0,S register claim,start,1970-01-01 07:20:25.686354+00:00,NaN,NaN,NaN,NaN,Call Centre Agent Sydney
4,0,S register claim,complete,1970-01-01 07:39:09.158755+00:00,NaN,NaN,NaN,NaN,Call Centre Agent Sydney
...,...,...,...,...,...,...,...,...,...
43429,3511,initiate payment,complete,1970-01-01 08:14:10.238212+00:00,NaN,NaN,NaN,NaN,Claims handler
43430,3511,close claim,start,1970-01-01 08:14:10.772015+00:00,NaN,NaN,NaN,NaN,Claims handler
43431,3511,close claim,complete,1970-01-01 08:16:27.040050+00:00,NaN,NaN,NaN,NaN,Claims handler
43432,3511,end,start,1970-01-01 08:16:26.501605+00:00,NaN,NaN,17508,processed,Claims handler


In [15]:
generator.event_attributes_info_df()

,name,level,xes_type,presence,numeralicity,is_trace_identifier
0,call centre,AttributeLevelEnum.TRACE,XesTypeEnum.STRING,0.076119,None,False
1,org:resource,AttributeLevelEnum.EVENT,XesTypeEnum.STRING,1.000000,None,False
2,time:timestamp,None,XesTypeEnum.DATE,NaN,None,None
3,lifecycle:transition,None,XesTypeEnum.STRING,NaN,None,None
4,concept:name,None,XesTypeEnum.STRING,NaN,None,None
5,location,AttributeLevelEnum.TRACE,XesTypeEnum.STRING,0.152239,None,False
6,duration,AttributeLevelEnum.TRACE,XesTypeEnum.STRING,0.152239,None,False
7,outcome,AttributeLevelEnum.TRACE,XesTypeEnum.STRING,0.152239,None,False


In [1]:
for ai in generator.event_attributes_info.values():
    print(ai.name)

attribute_info = next(ai for ai in generator.event_attributes_info.values() if ai.name == "org:resource")
attribute_info.name

NameError: name 'generator' is not defined

In [26]:
generator._log_df

,case:concept:name,concept:name,lifecycle:transition,time:timestamp
0,0,incoming claim,complete,1970-01-01 00:36:53.983009+00:00
1,0,S check if sufficient information is available,start,1970-01-01 00:37:15.860861+00:00
2,0,S check if sufficient information is available,complete,1970-01-01 00:37:39.280324+00:00
3,0,S register claim,start,1970-01-01 00:37:42.255830+00:00
4,0,S register claim,complete,1970-01-01 00:45:41.869864+00:00
...,...,...,...,...
12,3511,close claim,complete,1970-01-01 01:27:47.775616+00:00
13,3511,advise claimant on reimbursement,start,1970-01-01 01:27:48.412951+00:00
14,3511,advise claimant on reimbursement,complete,1970-01-01 01:30:56.866505+00:00
15,3511,end,start,1970-01-01 01:30:56.018827+00:00


In [18]:
new_df = attribute_info.predictor.generate(generator._log_df)
new_df

,case:concept:name,concept:name,lifecycle:transition,time:timestamp,org:resource
0,0,incoming claim,complete,1970-01-01 00:01:43.995412+00:00,customer
1,0,B check if sufficient information is available,start,1970-01-01 00:04:47.340217+00:00,Call Centre Agent Brisbane
2,0,B check if sufficient information is available,complete,1970-01-01 00:05:18.689928+00:00,Call Centre Agent Brisbane
3,0,B register claim,start,1970-01-01 00:05:19.518574+00:00,Call Centre Agent Brisbane
4,0,B register claim,complete,1970-01-01 00:11:20.318405+00:00,Call Centre Agent Brisbane
...,...,...,...,...,...
0,3511,incoming claim,complete,1970-01-01 07:46:41.789650+00:00,customer
1,3511,B check if sufficient information is available,start,1970-01-01 08:06:37.909918+00:00,Call Centre Agent Brisbane
2,3511,B check if sufficient information is available,complete,1970-01-01 08:06:44.478902+00:00,Call Centre Agent Brisbane
3,3511,end,start,1970-01-01 08:06:43.778741+00:00,Claims handler


In [20]:
df = generator._log_df.copy(deep=True)
df["org:resource"] = new_df['org:resource']
df

,case:concept:name,concept:name,lifecycle:transition,time:timestamp,org:resource
0,0,incoming claim,complete,1970-01-01 00:01:43.995412+00:00,customer
1,0,B check if sufficient information is available,start,1970-01-01 00:04:47.340217+00:00,Call Centre Agent Brisbane
2,0,B check if sufficient information is available,complete,1970-01-01 00:05:18.689928+00:00,Call Centre Agent Brisbane
3,0,B register claim,start,1970-01-01 00:05:19.518574+00:00,Call Centre Agent Brisbane
4,0,B register claim,complete,1970-01-01 00:11:20.318405+00:00,Call Centre Agent Brisbane
...,...,...,...,...,...
0,3511,incoming claim,complete,1970-01-01 07:46:41.789650+00:00,customer
1,3511,B check if sufficient information is available,start,1970-01-01 08:06:37.909918+00:00,Call Centre Agent Brisbane
2,3511,B check if sufficient information is available,complete,1970-01-01 08:06:44.478902+00:00,Call Centre Agent Brisbane
3,3511,end,start,1970-01-01 08:06:43.778741+00:00,Claims handler


In [7]:
from sklearn.model_selection import train_test_split

In [49]:
%autoreload

from maxes.generators.xes_generator.attribute_generators import AttributeGenerator, AttributePredictor_MLPClassifier

X_train_df, X_test_df = train_test_split(log.df, random_state=1)
print(X_train_df.shape)
print(X_test_df.shape)

inner_model = AttributePredictor_MLPClassifier(random_state=1)
model_for_org_resource = AttributeGenerator(input_columns=["concept:name"], output_column="org:resource", model=inner_model)

model_for_org_resource.fit(X_train_df)

# def evaluate_accuracy

(34603, 10)
(11535, 10)


In [50]:
X_test, y_test = model_for_org_resource.split_xy(X_test_df)
print(y_test.shape)

(11535,)


In [43]:
y_pred = model_for_org_resource.predict(X_test)
y_pred.shape

(11535,)

In [45]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.976159514521023

In [52]:
model_for_org_resource.score(X_test_df)

0.976159514521023

In [46]:
def score(model, X, y):
    # X, y = model.split_xy(df)

    inner_model = model.model

    X = inner_model.input_encoder.transform(X)

    y = np.array(y).reshape(-1, 1)
    y = inner_model.output_encoder.transform(y)

    return inner_model.model.score(X, y)

score(model_for_org_resource, X_test, y_test)

# model_for_org_resource.model.model.score(X_test, y_test)

0.976159514521023

In [97]:

from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [120]:
log.df = log.df.fillna('')
log.df

,case:concept:name,concept:name,lifecycle:transition,time:timestamp,call centre,org:resource,location,duration,outcome,case:description
0,1,incoming claim,complete,1970-01-01 00:00:00+00:00,Brisbane,customer,,,,Simulated process instance
1,1,B check if sufficient information is available,start,1970-01-01 00:00:00+00:00,,Call Centre Agent Brisbane,Brisbane,,,Simulated process instance
2,1,B check if sufficient information is available,complete,1970-01-01 00:00:41+00:00,,Call Centre Agent Brisbane,Brisbane,,,Simulated process instance
3,1,B register claim,start,1970-01-01 00:00:41+00:00,,Call Centre Agent Brisbane,,,,Simulated process instance
4,1,B register claim,complete,1970-01-01 00:12:59+00:00,,Call Centre Agent Brisbane,,,,Simulated process instance
...,...,...,...,...,...,...,...,...,...,...
4,3512,S register claim,complete,1970-01-01 02:39:31+00:00,,Call Centre Agent Sydney,,,,Simulated process instance
5,3512,determine likelihood of claim,start,1970-01-01 06:56:04+00:00,,Claims handler,,,,Simulated process instance
6,3512,determine likelihood of claim,complete,1970-01-01 06:56:35+00:00,,Claims handler,,,,Simulated process instance
7,3512,end,start,1970-01-01 06:56:35+00:00,,Claims handler,,17306,not liable,Simulated process instance


In [ ]:
%autoreload

from maxes.generators.xes_generator.attribute_generators import (
    AttributePredictor_MLPClassifier,
    AttributePredictor_CategoricalNB,
    AttributePredictor_KNeighborsClassifier,
    AttributePredictor_DecisionTreeClassifier,
    AttributePredictor_RandomForestClassifier,
    AttributeGenerator
)

def evaluate_accuracy(model, df: pd.DataFrame, random_state=None):
    X_train, X_test = train_test_split(df, random_state=random_state)

    print(X_train.shape)
    print(X_test.shape)

    model.fit(X_train)

    X_test, y_test = model.split_xy(X_test)
    y_test = y_test.astype('category')

    y_pred = model.predict(X_test)
    y_pred = pd.Series(y_pred).astype('category')

    return accuracy_score(y_test, y_pred, normalize=True)

# print("MLPClassifier")
# accuracy = evaluate_accuracy(
#     model=AttributeGenerator(
#         input_columns=["concept:name"],
#         output_column="org:resource",
#         model=AttributePredictor_MLPClassifier(random_state=1)
#     ),
#     df=log.df,
#     random_state=1
# )
# print(accuracy)

# print("CategoricalNB")
# accuracy = evaluate_accuracy(
#     model=AttributeGenerator(
#         input_columns=["concept:name"],
#         output_column="org:resource",
#         model=AttributePredictor_CategoricalNB(random_state=1)
#     ),
#     df=log.df,
#     random_state=1
# )
# print(accuracy)

# print("KNeighborsClassifier")
# accuracy = evaluate_accuracy(
#     model=AttributeGenerator(
#         input_columns=["concept:name"],
#         output_column="org:resource",
#         model=AttributePredictor_KNeighborsClassifier(random_state=1)
#     ),
#     df=log.df,
#     random_state=1
# )
# print(accuracy)

# print("DecisionTreeClassifier")
# accuracy = evaluate_accuracy(
#     model=AttributeGenerator(
#         input_columns=["concept:name"],
#         output_column="org:resource",
#         model=AttributePredictor_DecisionTreeClassifier(random_state=1)
#     ),
#     df=log.df,
#     random_state=1
# )
# print(accuracy)

# print("RandomForestClassifier")
# accuracy = evaluate_accuracy(
#     model=AttributeGenerator(
#         input_columns=["concept:name"],
#         output_column="org:resource",
#         model=AttributePredictor_RandomForestClassifier(random_state=1)
#     ),
#     df=log.df,
#     random_state=1
# )
# print(accuracy)

In [122]:
%autoreload

from maxes.generators.xes_generator.attribute_generators import AttributePredictor_ClassifierWrapper

inner_model = AttributePredictor_ClassifierWrapper(
    random_state=1,
    inner_model_class=CategoricalNB,
    inner_model_kwargs={}
)
model_for_org_resource = AttributeGenerator(input_columns=["concept:name"], output_column="location", model=inner_model)

evaluate_accuracy(
    model=model_for_org_resource,
    df=log.df,
    random_state=1
)

(34603, 10)
(11535, 10)


1.0

In [ ]:
%autoreload

from sklearn.model_selection import cross_val_score
from maxes.generators.xes_generator.attribute_generators import (
    AttributeGenerator,
    AttributePredictor_MLPClassifier,
    AttributePredictor_CategoricalNB,
    AttributePredictor_KNeighborsClassifier,
    AttributePredictor_DecisionTreeClassifier,
    AttributePredictor_RandomForestClassifier,
)

def evaluate_cross_val_accuracy(model, df: pd.DataFrame, random_state=None):
    inner_model = model.model
    X, y = model.split_xy(df)

    return cross_val_score(inner_model, X, y, cv=5)

print("MLPClassifier")
scores = evaluate_cross_val_accuracy(
    model=AttributeGenerator(
        input_columns=["concept:name"],
        output_column="org:resource",
        model=AttributePredictor_MLPClassifier(random_state=1)
    ),
    df=log.df,
    random_state=1
)
print(scores)
print(scores.mean())

print("CategoricalNB")
scores = evaluate_cross_val_accuracy(
    model=AttributeGenerator(
        input_columns=["concept:name"],
        output_column="org:resource",
        model=AttributePredictor_CategoricalNB(random_state=1)
    ),
    df=log.df,
    random_state=1
)
print(scores)
print(scores.mean())

print("KNeighborsClassifier")
scores = evaluate_cross_val_accuracy(
    model=AttributeGenerator(
        input_columns=["concept:name"],
        output_column="org:resource",
        model=AttributePredictor_KNeighborsClassifier(random_state=1)
    ),
    df=log.df,
    random_state=1
)
print(scores)
print(scores.mean())

print("DecisionTreeClassifier")
scores = evaluate_cross_val_accuracy(
    model=AttributeGenerator(
        input_columns=["concept:name"],
        output_column="org:resource",
        model=AttributePredictor_DecisionTreeClassifier(random_state=1)
    ),
    df=log.df,
    random_state=1
)
print(scores)
print(scores.mean())

print("RandomForestClassifier")
scores = evaluate_cross_val_accuracy(
    model=AttributeGenerator(
        input_columns=["concept:name"],
        output_column="org:resource",
        model=AttributePredictor_RandomForestClassifier(random_state=1)
    ),
    df=log.df,
    random_state=1
)
print(scores)
print(scores.mean())

MLPClassifier
[0.97550932 0.97659298 0.97529259 0.97507315 0.97572342]
0.9756382920800881
CategoricalNB
[0.97550932 0.97659298 0.97529259 0.97507315 0.97572342]
0.9756382920800881
KNeighborsClassifier
[0.97550932 0.97659298 0.97529259 0.97507315 0.97572342]
0.9756382920800881
DecisionTreeClassifier
[0.97550932 0.97659298 0.97529259 0.97507315 0.97572342]
0.9756382920800881
RandomForestClassifier
[0.97550932 0.97659298 0.97529259 0.97507315 0.97572342]
0.9756382920800881


In [125]:
%autoreload

from sklearn.model_selection import cross_val_score
from maxes.generators.xes_generator.attribute_generators import (
    AttributeGenerator,
    AttributePredictor_ClassifierWrapper
)

from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

inner_model_classes = [
    MLPClassifier,
    CategoricalNB,
    KNeighborsClassifier,
    DecisionTreeClassifier,
    RandomForestClassifier,
]

columns = [
    "org:resource",
    "location",
    "call centre",
    "outcome",
]

for column in columns:
    for inner_model_class in inner_model_classes:
        print(f"column: {column}")
        print(f"model: {inner_model_class}")

        predictor_model = AttributePredictor_ClassifierWrapper(
            random_state=1,
            inner_model_class=inner_model_class,
            inner_model_kwargs={}
        )

        model = AttributeGenerator(
            input_columns=["concept:name"],
            output_column="location",
            model=predictor_model
        )

        score = evaluate_cross_val_accuracy(
            model=model,
            df=log.df,
            random_state=1
        )

        print(score)

column: org:resource
model: <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>
[1. 1. 1. 1. 1.]
column: org:resource
model: <class 'sklearn.naive_bayes.CategoricalNB'>
[1. 1. 1. 1. 1.]
column: org:resource
model: <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
[1. 1. 1. 1. 1.]
column: org:resource
model: <class 'sklearn.tree._classes.DecisionTreeClassifier'>
[1. 1. 1. 1. 1.]
column: org:resource
model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
[1. 1. 1. 1. 1.]
column: location
model: <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>
[1. 1. 1. 1. 1.]
column: location
model: <class 'sklearn.naive_bayes.CategoricalNB'>
[1. 1. 1. 1. 1.]
column: location
model: <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
[1. 1. 1. 1. 1.]
column: location
model: <class 'sklearn.tree._classes.DecisionTreeClassifier'>
[1. 1. 1. 1. 1.]
column: location
model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
[1.